In [ ]:
# Banana Feature Prediction Model

This notebook implements a machine learning model to predict:
1. The number of seeds in a banana
2. The curvature of the banana (in degrees)

The predictions are based on various physical features. The model will be trained using a labeled dataset (to be added later).

# Dataset Requirements

## Image Data
- Format: JPG or PNG
- Resolution: 224x224 pixels (standard for many vision models)
- Perspective: Side view of the banana
- Background: Preferably solid color or simple background
- Lighting: Consistent, well-lit conditions
- File naming convention: `banana_[id].jpg`

## Labels Dataset (CSV format)
Required columns:
1. `image_filename` (string): Name of the corresponding image file
2. `length_cm` (float): Length of the banana in centimeters
3. `width_cm` (float): Width at the middle point in centimeters
4. `weight_g` (float): Weight in grams
5. `ripeness` (int): Scale from 1-5 where:
   - 1: Very unripe (green)
   - 2: Unripe (more green than yellow)
   - 3: Ripe (yellow with few brown spots)
   - 4: Very ripe (yellow with many brown spots)
   - 5: Overripe (mostly brown)
6. `color_code` (int): 
   - 1: Green
   - 2: Yellow
   - 3: Brown
7. `seed_count` (int): Actual number of seeds
8. `curvature_degrees` (float): Measured angle of curvature

Example CSV format:
```csv
image_filename,length_cm,width_cm,weight_g,ripeness,color_code,seed_count,curvature_degrees
banana_001.jpg,15.2,3.1,125.5,3,2,8,45.2
banana_002.jpg,14.8,2.9,118.3,4,2,12,38.7
```

## Measurement Guidelines
1. Length: Measure along the outer curve from tip to tip
2. Width: Measure at the widest point
3. Curvature: Measure the angle between the stem and tip
4. Seeds: Count after careful dissection
5. Weight: Measure with peeled banana

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Additional imports for image processing and deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import cv2
from pathlib import Path

In [3]:
# Define image preprocessing
def preprocess_image(image_path):
    """Preprocess image for the model"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

# Define the CNN model
class BananaNet(nn.Module):
    def __init__(self):
        super(BananaNet, self).__init__()
        # Use pretrained ResNet18 as base model
        self.base_model = models.resnet18(pretrained=True)
        
        # Modify the final layer for our regression tasks
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()  # Remove final layer
        
        # Add custom layers for our specific predictions
        self.regression_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)  # 2 outputs: seed_count and curvature
        )
        
    def forward(self, x):
        features = self.base_model(x)
        output = self.regression_head(features)
        return output  # Returns [seed_count, curvature]

In [4]:
# Function to make predictions from image
def predict_from_image(model, image_path):
    """
    Predict seed count and curvature from a banana image
    
    Args:
        model: Trained BananaNet model
        image_path: Path to the banana image
        
    Returns:
        dict: Predictions for seed count and curvature
    """
    # Preprocess image
    image_tensor = preprocess_image(image_path)
    
    # Set model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Get predictions
        predictions = model(image_tensor)
        
        # Extract predictions
        seed_count = int(round(predictions[0][0].item()))
        curvature = round(predictions[0][1].item(), 1)
        
        return {
            'seeds': max(0, seed_count),  # Ensure non-negative
            'curvature': max(20, min(70, curvature))  # Clip to valid range
        }

In [ ]:
# Load and prepare the dataset
class BananaDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = Path(img_dir)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the image path and join it correctly with the base directory
        relative_img_path = self.data.iloc[idx]['image_filename']
        img_path = self.img_dir / relative_img_path
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            raise
        
        if self.transform:
            image = self.transform(image)
        
        # Get labels
        seed_count = self.data.iloc[idx]['seed_count']
        curvature = self.data.iloc[idx]['curvature_degrees']
        
        # Scale targets to new ranges for training using real min/max from the dataset
        # This ensures the model output matches the true range of your data and improves learning.
        seed_min, seed_max = 100, 450  # Realistic min/max for your dataset
        curv_min, curv_max = 170, 290  # Realistic min/max for your dataset
        seed_count = 350 + ((seed_count - seed_min) / (seed_max - seed_min)) * 100
        curvature = 180 + ((curvature - curv_min) / (curv_max - curv_min)) * 100
        
        labels = torch.tensor([seed_count, curvature], dtype=torch.float32)
        
        return image, labels

# Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [6]:
# Verify and clean dataset
print("Verifying dataset...")
df = pd.read_csv('training_data/dataset.csv')
valid_images = []
image_dir = Path('training_data')

for idx, row in df.iterrows():
    img_path = image_dir / row['image_filename']
    if img_path.exists():
        valid_images.append(row)
    else:
        print(f"Warning: Image not found: {img_path}")

# Create clean dataset
clean_df = pd.DataFrame(valid_images)
print(f"\nFound {len(clean_df)} valid images out of {len(df)} entries")
clean_df.to_csv('training_data/clean_dataset.csv', index=False)
print("Saved cleaned dataset to training_data/clean_dataset.csv")

Verifying dataset...

Found 31 valid images out of 32 entries
Saved cleaned dataset to training_data/clean_dataset.csv


In [8]:
# Create data loaders with proper error handling
try:
    # Print dataset info for debugging
    print("Loading cleaned dataset...")
    dataset = BananaDataset(
        csv_file='training_data/clean_dataset.csv',
        img_dir='training_data',
        transform=train_transform
    )
    
    # Split into train and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    print(f"\nSplit dataset into {train_size} training and {val_size} validation samples")
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=4,  # Reduced batch size
        shuffle=True,
        num_workers=0  # Disabled multiprocessing for debugging
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset, 
        batch_size=4,  # Reduced batch size
        shuffle=False,
        num_workers=0  # Disabled multiprocessing for debugging
    )
    
    # Initialize model, loss function, and optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BananaNet().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 200  # Increased from 100 to 200 for further training
    best_val_loss = float('inf')
    
    print(f"\nTraining on {device}")
    print("=" * 50)
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 40)
        
        # Training phase
        model.train()
        train_loss = 0.0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if (batch_idx + 1) % 2 == 0:
                print(f"    Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        train_loss = train_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        val_loss = val_loss / len(val_loader)
        
        print(f"\nTraining Loss: {train_loss:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved new best model!")
        
        print("=" * 50)
        
except Exception as e:
    print(f"Error during training: {str(e)}")
    import traceback
    traceback.print_exc()

Loading cleaned dataset...

Split dataset into 24 training and 7 validation samples

Training on cpu

Epoch 1/100
----------------------------------------


c:\Users\amalb\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\amalb\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


    Batch 2/6, Loss: 109269.8203
    Batch 4/6, Loss: 104654.4922
    Batch 4/6, Loss: 104654.4922
    Batch 6/6, Loss: 113989.2031

Training Loss: 109183.0417
Validation Loss: 106948.2109
Saved new best model!

Epoch 2/100
----------------------------------------
    Batch 6/6, Loss: 113989.2031

Training Loss: 109183.0417
Validation Loss: 106948.2109
Saved new best model!

Epoch 2/100
----------------------------------------
    Batch 2/6, Loss: 104122.2266
    Batch 2/6, Loss: 104122.2266
    Batch 4/6, Loss: 90400.1562
    Batch 4/6, Loss: 90400.1562
    Batch 6/6, Loss: 84159.4688

Training Loss: 95006.3828
Validation Loss: 74246.2109
Saved new best model!

Epoch 3/100
----------------------------------------
    Batch 6/6, Loss: 84159.4688

Training Loss: 95006.3828
Validation Loss: 74246.2109
Saved new best model!

Epoch 3/100
----------------------------------------
    Batch 2/6, Loss: 67734.2734
    Batch 2/6, Loss: 67734.2734
    Batch 4/6, Loss: 57842.9609
    Batch 4/6, Lo

In [ ]:
# Function to test the model on a single image
def test_prediction(model, image_path, device):
    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Get predictions
    model.eval()
    with torch.no_grad():
        predictions = model(image_tensor)
        
        # Denormalize predictions
        seed_count = int(round(predictions[0][0].item() * 500))  # Denormalize seeds
        curvature = round(predictions[0][1].item() * 360, 1)    # Denormalize degrees
        
        return {
            'seeds': max(0, seed_count),
            'curvature': max(0, min(360, curvature))
        }

# Test the model on a sample image
# Note: Replace 'sample_image.jpg' with an actual image path
print("Model training complete! You can now use test_prediction() to make predictions on new images.")

# Data Preparation

The model will use the following input features to make predictions:
1. Length (cm)
2. Width (cm)
3. Weight (g)
4. Ripeness level (1-5)
5. Color (encoded)

The model will predict:
1. Number of seeds
2. Curvature (degrees)

In [6]:
# Create sample data structure (to be replaced with real data later)
def create_sample_data(n_samples=100):
    np.random.seed(42)
    
    # Generate synthetic features
    length = np.random.normal(15, 2, n_samples)  # Average banana length 15cm
    width = np.random.normal(3, 0.5, n_samples)  # Average banana width 3cm
    weight = np.random.normal(120, 20, n_samples)  # Average banana weight 120g
    ripeness = np.random.randint(1, 6, n_samples)  # Ripeness level 1-5
    color = np.random.randint(1, 4, n_samples)  # Color encoded as 1=green, 2=yellow, 3=brown
    
    # Create feature matrix
    X = np.column_stack([length, width, weight, ripeness, color])
    
    # Generate targets
    # 1. Number of seeds
    seeds = (0.5 * length + 0.3 * width + 0.2 * weight/100 + 0.1 * ripeness + 
            0.1 * color).astype(int)
    seeds = np.clip(seeds, 0, 20)  # Limit number of seeds between 0 and 20
    
    # 2. Curvature (degrees) - influenced by length and ripeness
    curvature = (45 + 0.5 * (length - 15) + 2 * (ripeness - 3) + 
                 np.random.normal(0, 5, n_samples))
    curvature = np.clip(curvature, 20, 70)  # Limit curvature between 20 and 70 degrees
    
    # Create DataFrame for features
    columns = ['length', 'width', 'weight', 'ripeness', 'color']
    df = pd.DataFrame(X, columns=columns)
    
    # Stack targets into a single array
    y = np.column_stack([seeds, curvature])
    
    return df, y

# Create sample dataset
X_data, y_data = create_sample_data()

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the model
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42
)
model.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [8]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)

# Calculate metrics for both predictions
# Seeds prediction
mse_seeds = mean_squared_error(y_test[:, 0], y_pred[:, 0])
r2_seeds = r2_score(y_test[:, 0], y_pred[:, 0])

# Curvature prediction
mse_curvature = mean_squared_error(y_test[:, 1], y_pred[:, 1])
r2_curvature = r2_score(y_test[:, 1], y_pred[:, 1])

print("Seeds Prediction:")
print(f"Mean Squared Error: {mse_seeds:.2f}")
print(f"R² Score: {r2_seeds:.2f}")
print("\nCurvature Prediction:")
print(f"Mean Squared Error: {mse_curvature:.2f}")
print(f"R² Score: {r2_curvature:.2f}")

Seeds Prediction:
Mean Squared Error: 0.14
R² Score: 0.83

Curvature Prediction:
Mean Squared Error: 21.98
R² Score: 0.27


In [9]:
# Function to predict seeds and curvature for new bananas
def predict_banana_features(length, width, weight, ripeness, color):
    # Create a feature array for the new banana
    features = np.array([[length, width, weight, ripeness, color]])
    
    # Scale the features
    features_scaled = scaler.transform(features)
    
    # Make predictions
    predictions = model.predict(features_scaled)[0]
    
    return {
        'seeds': int(predictions[0]),
        'curvature': round(predictions[1], 1)
    }

# Example prediction
example_banana = {
    'length': 16,    # cm
    'width': 3.2,    # cm
    'weight': 130,   # g
    'ripeness': 4,   # scale 1-5
    'color': 2,      # yellow
}

predictions = predict_banana_features(
    example_banana['length'],
    example_banana['width'],
    example_banana['weight'],
    example_banana['ripeness'],
    example_banana['color']
)

print(f"Predicted number of seeds: {predictions['seeds']}")
print(f"Predicted curvature: {predictions['curvature']}°")

Predicted number of seeds: 9
Predicted curvature: 49.9°


c:\Users\Icarus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
